In [3]:
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import odeint
import math
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools


#import csvs
train = pd.read_csv("../../data/train.csv")
test = pd.read_csv("../../data/test.csv")
graph = pd.read_csv("../../data/graph.csv")

statesdata = {}
states = pd.Series.unique(train['Province_State'])
num_states = len(states)
for s in states:
    statesdata[s] = train.loc[train['Province_State'] == s ,:]

In [4]:

days = 26

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

def predictARIMA(X, p, d, q, days, sarimax=False):
    start = len(X)
    if sarimax:
        # model = SARIMAX(X, order=(p,d,q), measurement_error=True).fit(disp=False)
        model = SARIMAX(X,order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False).fit(disp=False)
        forecast = model.forecast(days)
    else:
        model = ARIMA(X, order=(p,d,q)).fit()
        forecast = model_fit.predict(start = start, end = start + days)
    return(forecast)

def bestARIMA(X):
    p=d=q=range(0,6)
    a=99999
    pdq=list(itertools.product(p,d,q))

    for var in pdq:
        try:
            model = ARIMA(X, order=var)
            result = model.fit()

            if (result.aic<=a) :
                a=result.aic
                param=var
        except:
            continue
    
    model = ARIMA(X, order=param).fit()
    forecast = model.predict(start = len(X), end = len(X)+26)
    return forecast, param

    
proj = {}
for s in states:
    a = statesdata[s]
    a = a.reset_index()
    confirmed = a['Confirmed']
    deaths = a['Deaths']
    
    X, Y = confirmed.values, deaths.values
    
    forecastC, param_C = bestARIMA(X)
    forecastD, param_D = bestARIMA(Y)
    print(param_C, param_D)
    # forecastC = predictARIMA(X, 3,2,1, days) # BEST PARAMS SO FAR ARE 3,2,1
    # forecastD = predictARIMA(Y, 4,2,3, days) # BEST PARAMS SO FAR ARE 4,2,3

    df = {'Confirmed': forecastC, 'Deaths': forecastD}
    
    proj[s] = pd.DataFrame(df)

(2, 0, 3) (3, 0, 0)
(5, 2, 4) (2, 2, 3)
(2, 5, 3) (3, 0, 3)
(1, 4, 4) (2, 2, 4)
(5, 5, 2) (3, 0, 0)
(4, 2, 2) (4, 2, 5)
(2, 1, 4) (5, 3, 5)
(2, 0, 4) (0, 2, 1)
(0, 4, 5) (5, 2, 4)
(5, 3, 5) (4, 2, 5)


In [1]:
order = test.loc[0:49,'Province_State']

# format submission
conf = []
dead = []
fid = 0
for i in range(days):
    for j in order:
        projection = proj[j].iloc[i]
        #print(j, 'day', i)
        conf.append(int(projection['Confirmed']))
        dead.append(int(projection['Deaths']))
        #print(fid)
        fid+=1

test['Confirmed'] = conf
test['Deaths'] = dead

NameError: name 'test' is not defined

In [64]:
submission = test.drop(columns=['Province_State', 'Date'])
print(submission)
submission.to_csv('team25.csv', index = False, header = True)

ForecastID  Confirmed  Deaths
0              0     127415    2202
1              1       5299      37
2              2     202195    5059
3              3      61771     811
4              4     717025   13116
...          ...        ...     ...
1295        1295     144320    2966
1296        1296      87723    2144
1297        1297      13715     306
1298        1298      92154    1271
1299        1299       4631      44

[1300 rows x 3 columns]


In [65]:
def give_mape(ground_truth_path, prediction_path):
    ground_truth = pd.read_csv(ground_truth_path)
    predictions = pd.read_csv(prediction_path)

    gt_conf = ground_truth['Confirmed']
    gt_dead = ground_truth['Deaths']

    pred_conf = predictions['Confirmed']
    pred_dead = predictions['Deaths']

    total = 0
    for i in range(1300):
        conf_error = float(pred_conf[i] - gt_conf[i]) / float(gt_conf[i])
        dead_error = float(pred_dead[i] - gt_dead[i]) / float(gt_dead[i])
        total = total + abs(conf_error) + abs(dead_error)

    total = total / 2600
    return total

print(give_mape('team25.csv', '../ValidationTester/ground_truth.csv'))

0.029686984634018823
